In [3]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline

import torch
from torch import nn
import torch.nn.functional as F

from detection_utils.boxes import non_max_suppression, generate_targets
from detection_utils.metrics import compute_recall, compute_precision
from detection_utils.pytorch import softmax_focal_loss

import cv2

In [ ]:
# here we'll define a simple helper function to load a dataset
# then load our training and validation sets
def load_data(root_dir):
    root_dir = Path(root_dir)
    images = np.load(root_dir / 'images.npy')
    # we're loading object arrays here so we need to allow_pickle
    boxes = np.load(root_dir / 'boxes.npy', allow_pickle=True)
    labels = np.load(root_dir / 'labels.npy', allow_pickle=True)
    return images, boxes, labels

train_images, train_boxes, train_labels = load_data('data/train/')
val_images, val_boxes, val_labels = load_data('data/val/')

# for performance, we'll load all of our images onto our device ahead of time to minimize
# data transfers from CPU -> GPU. We know we're working with a small dataset, so this shouldn't
# cause problems. This uses about a gigabyte of GPU memory.
# We'll do the transpose to convert our images from shape (N, 256, 256, 3) to (N, 3, 256, 256)
# which is the ordering we need for our CNN.
train_images = torch.tensor(train_images.transpose(0, 3, 1, 2)).to(device)
val_images = torch.tensor(val_images.transpose(0, 3, 1, 2)).to(device)